# Train faster, more flexible models with Amazon SageMaker Linear Learner (Binary Classifier)

## Background
This notebook is part of a series of notebooks whose purpose is to demonstrate additional features implemented within the built-in SageMaker Linear Learner algorithm, namely early stopping, loss functions, and class weights. If you want more theoretical background behind these features, please see the [README.md](README.md). This is the first notebook in the series, which demonstrates the use of these features with the default binary classifier. You can choose to run this notebook by itself or in sequence with the other notebooks listed below. 

1. [Train faster, more flexible models with Amazon SageMaker Linear Learner (Binary Classifier)](linear_learner_class_weights_loss_functions_default.ipynb) (current notebook)
1. [Train faster, more flexible models with Amazon SageMaker Linear Learner (SVM)](linear_learner_class_weights_loss_functions_svm.ipynb)


## Hands-on example: Detecting credit card fraud

In this section, we'll look at a credit card fraud detection dataset.  The data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data).  We have features and labels for over a quarter million credit card transactions, each of which is labeled as fraudulent or not fraudulent.  We'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.  This is a binary classification problem.  

We'll walk through training linear learner with various settings and deploying an inference endpoint.  We'll evaluate the quality of our models by hitting that endpoint with observations from the test set.  We can take the real-time predictions returned by the endpoint and evaluate them against the ground-truth labels in our test set.

Next, we'll apply the linear learner threshold tuning functionality to get better precision without sacrificing recall.  Then, we'll push the precision even higher using the linear learner new class weights feature.  Because fraud can be extremely costly, we would prefer to have high recall, even if this means more false positives.  This is especially true if we are building a first line of defense, flagging potentially fraudulent transactions for further review before taking actions that affect customers.

First we'll do some preprocessing on this data set: we'll shuffle the examples and split them into train and test sets.  To run this under notebook under your own AWS account, you'll need to change the Amazon S3 locations.  First download the raw data from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data) and upload to your SageMaker notebook instance (or wherever you're running this notebook).  Only 0.17% of the data have positive labels, making this a challenging classification problem.

In [ ]:
import boto3
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_deserializer

In [ ]:
! aws s3 cp s3://sagemaker-sample-files/datasets/tabular/mlg-ulb-credit-card/creditcard_csv.csv .

In [ ]:
# Set data locations
bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-linear-learner-loss-weights"  # replace this with your own prefix
s3_train_key = "{}/train/recordio-pb-data".format(prefix)
s3_train_path = os.path.join("s3://", bucket, s3_train_key)
local_raw_data = "creditcard_csv.csv"
role = get_execution_role()

In [ ]:
# Confirm access to s3 bucket
for obj in boto3.resource("s3").Bucket(bucket).objects.all():
    print(obj.key)

In [ ]:
# Read the data, shuffle, and split into train and test sets, separating the labels (last column) from the features
raw_data = pd.read_csv(local_raw_data).values
np.random.seed(0)
np.random.shuffle(raw_data)
train_size = int(raw_data.shape[0] * 0.7)
train_features = raw_data[:train_size, :-1]
train_labels = np.array([x.strip("'") for x in raw_data[:train_size, -1]]).astype(np.int)
test_features = raw_data[train_size:, :-1]
test_labels = np.array([x.strip("'") for x in raw_data[train_size:, -1]]).astype(np.int)


# Convert the processed training data to protobuf and write to S3 for linear learner
vectors = np.array([t.tolist() for t in train_features]).astype("float32")
labels = np.array([t.tolist() for t in train_labels]).astype("float32")
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)
boto3.resource("s3").Bucket(bucket).Object(s3_train_key).upload_fileobj(buf)

We'll wrap the model training setup in a convenience function that takes in the S3 location of the training data, the model hyperparameters that define our training job, and the S3 output path for model artifacts.  Inside the function, we'll hardcode the algorithm container, the number and type of EC2 instances to train on, and the input and output data formats.

In [ ]:
from utils import predictor_from_hyperparams

And import another convenience function for setting up a hosting endpoint, making predictions, and evaluating the model.  To make predictions, we need to set up a model hosting endpoint.  Then we feed test features to the endpoint and receive predicted test labels.  To evaluate the models we create in this exercise, we'll capture predicted test labels and compare them to actuals using some common binary classification metrics.

In [ ]:
from utils import evaluate

And finally we'll import a convenience function to delete prediction endpoints after we're done with them:

In [ ]:
from utils import delete_endpoint

Let's begin by training a binary classifier model with the linear learner default settings.  Note that we're setting the number of epochs to 40, which is much higher than the default of 10 epochs.  With early stopping, we don't have to worry about setting the number of epochs too high.  Linear learner will stop training automatically after the model has converged.

In [ ]:
# Training a binary classifier with default settings: logistic regression
defaults_hyperparams = {"feature_dim": 30, "predictor_type": "binary_classifier", "epochs": 40}
defaults_output_path = "s3://{}/{}/defaults/output".format(bucket, prefix)
defaults_predictor = predictor_from_hyperparams(
    s3_train_path, defaults_hyperparams, defaults_output_path
)

And now we'll produce a model with a threshold tuned for the best possible precision with recall fixed at 90%:

In [ ]:
# Training a binary classifier with automated threshold tuning
autothresh_hyperparams = {
    "feature_dim": 30,
    "predictor_type": "binary_classifier",
    "binary_classifier_model_selection_criteria": "precision_at_target_recall",
    "target_recall": 0.9,
    "epochs": 40,
}
autothresh_output_path = "s3://{}/{}/autothresh/output".format(bucket, prefix)
autothresh_predictor = predictor_from_hyperparams(
    s3_train_path, autothresh_hyperparams, autothresh_output_path
)

### Improving recall with class weights

Now we'll improve on these results using a new feature added to linear learner: class weights for binary classification.  We introduced this feature in the *Class Weights* section, and now we'll look into its application to the credit card fraud dataset by training a new model with balanced class weights:

In [ ]:
# Training a binary classifier with class weights and automated threshold tuning
class_weights_hyperparams = {
    "feature_dim": 30,
    "predictor_type": "binary_classifier",
    "binary_classifier_model_selection_criteria": "precision_at_target_recall",
    "target_recall": 0.9,
    "positive_example_weight_mult": "balanced",
    "epochs": 40,
}
class_weights_output_path = "s3://{}/{}/class_weights/output".format(bucket, prefix)
class_weights_predictor = predictor_from_hyperparams(
    s3_train_path, class_weights_hyperparams, class_weights_output_path
)

Now we'll make use of the prediction endpoint we've set up for each model by sending them features from the test set and evaluating their predictions with standard binary classification metrics.

In [ ]:
# Evaluate the trained models
predictors = {
    "Logistic": defaults_predictor,
    "Logistic with auto threshold": autothresh_predictor,
    "Logistic with class weights": class_weights_predictor,
}
metrics = {
    key: evaluate(predictor, test_features, test_labels, key, False)
    for key, predictor in predictors.items()
}
pd.set_option("display.float_format", lambda x: "%.3f" % x)
display(
    pd.DataFrame(list(metrics.values())).loc[:, ["Model", "Recall", "Precision", "Accuracy", "F1"]]
)

The results are in!  With threshold tuning, we can accurately predict 85-90% of the fraudulent transactions in the test set (due to randomness in training, recall will vary between 0.85-0.9 across multiple runs).  But in addition to those true positives, we'll have a high number of false positives: 90-95% of the transactions we predict to be fraudulent are in fact not fraudulent (precision varies between 0.05-0.1).  This model would work well as a first line of defense, flagging potentially fraudulent transactions for further review.  If we instead want a model that gives very few false alarms, at the cost of catching far fewer of the fraudulent transactions, then we should optimize for higher precision:

    binary_classifier_model_selection_criteria='recall_at_target_precision', 
    target_precision=0.9,

And what about the results of using our new feature, class weights for binary classification?  Training with class weights has made a huge improvement to this model's performance!  The precision is roughly doubled, while recall is still held constant at 85-90%.  

Balancing class weights improved the performance of our SVM predictor, but it still does not match the corresponding logistic regression model for this dataset.  Comparing all of the models we've fit so far, logistic regression with class weights and tuned thresholds did the best.

#### Note on target vs. observed recall

It's worth taking some time to look more closely at these results.  If we asked linear learner for a model calibrated to a target recall of 0.9, then why didn't we get exactly 90% recall on the test set?  The reason is the difference between training, validation, and testing.  Linear learner calibrates thresholds for binary classification on the validation data set when one is provided, or else on the training set.  Since we did not provide a validation data set, the threshold were calculated on the training data.  Since the training, validation, and test data sets don't match exactly, the target recall we request is only an approximation.  In this case, the threshold that produced 90% recall on the training data happened to produce only 85-90% recall on the test data (due to some randomness in training, the results will vary from one run to the next).  The variation of recall in the test set versus the training set is dependent on the number of positive points. In this example, although we have over 280,000 examples in the entire dataset, we only have 337 positive examples, hence the large difference.  The accuracy of this approximation can be improved by providing a large validation data set to get a more accurate threshold, and then evaluating on a large test set to get a more accurate benchmark of the model and its threshold.  For even more fine-grained control, we can set the number of calibration samples to a higher number.  It's default value is already quite high at 10 million samples:

    num_calibration_samples=10000000,

#### Clean Up

Finally we'll clean up by deleting the prediction endpoints we set up:

In [ ]:
for predictor in [
    defaults_predictor,
    autothresh_predictor,
    class_weights_predictor,
]:
    delete_endpoint(predictor)

We've just shown how to use the linear learner new early stopping feature, new loss functions, and new class weights feature to improve credit card fraud prediction.  Class weights can help you optimize recall or precision for all types of fraud detection, as well as other classification problems with rare events, like ad click prediction or mechanical failure prediction.  Try using class weights in your binary classification problem, or try one of the new loss functions for your regression problems: use quantile prediction to put confidence intervals around your predictions by learning 5% and 95% quantiles.  For more information about new loss functions and class weights, see the linear learner [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html).

##### References
Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015.